In [1]:
from DRP.recommender.SampleGridParameters import GridValueGenerator

In [2]:
from DRP.recommender.ReactionGenerator import ReactionGenerator

In [3]:
a = GridValueGenerator().main()
react_gen = ReactionGenerator(*a)

In [4]:
react_gen

In [5]:
example_list = []

In [6]:
count = 0
for i in react_gen.generate():
    count+=1
    if count >= 100:
        break
    else:
        example_list.append(i)
    

<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.

In [7]:
from DRP.models import OrdRxnDescriptor

In [8]:
for i in OrdRxnDescriptor.objects.all():
    print i.pk, i

1 Four class crystallisation outcome
3 Two class crystallisation purity


In [9]:
from DRP.models import NumRxnDescriptorValue
from DRP.models import OrdRxnDescriptorValue

In [10]:
NumRxnDescriptorValue.objects.filter(reaction = example_list[0])

[<NumRxnDescriptorValue: NumRxnDescriptorValue object>, <NumRxnDescriptorValue: NumRxnDescriptorValue object>, <NumRxnDescriptorValue: NumRxnDescriptorValue object>]

In [11]:
initial_len = len(NumRxnDescriptorValue.objects.all())

In [12]:
count = 0
for i in react_gen.generate():
    count+=1
    if count >= 2:
        break
    else:
        example_list.append(i)
        
assert(initial_len < len(NumRxnDescriptorValue.objects.all()))

<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>
<class 'DRP.models.descriptors.NumericDescriptor'>


In [13]:
initial_len

46513

In [14]:
len(NumRxnDescriptorValue.objects.all())

46519

In [15]:
len(NumRxnDescriptorValue.objects.all())-initial_len

6

In [16]:
example_list[0:7]

[<Reaction: Reaction_161583>,
 <Reaction: Reaction_161584>,
 <Reaction: Reaction_161585>,
 <Reaction: Reaction_161586>,
 <Reaction: Reaction_161587>,
 <Reaction: Reaction_161588>,
 <Reaction: Reaction_161589>]

In [17]:
from DRP.models import OrdRxnDescriptorValue


class ReactionSieve(object):

    def __init__(self, model_container, desired_desc_dict):
      self.model_container = model_container
      self.desired_desc_dict = desired_desc_dict
    
    def filter(self, reactions):
        self.model_container.predict(reactions)
        for desc, desc_vals in self.desired_desc_dict.items():
          reactions = filter(lambda rxn: rxn.descriptor_values.filter(value__in=desc_vals,descriptor=desc).exists(), reactions)
        return reactions


In [18]:
from DRP.recommender.AbstractRecommender import AbstractRecommender

class Recommender(AbstractRecommender):
  def __init__(self, model_container, desc_dict, desired_desc_dict):
    """
    All Descriptor Dicts use (Descriptor, Value List) pairs, where the
    Value List is a list of any values this Descriptor should have.
    """
    self.generator = ReactionGenerator(desc_dict)
    self.sieve = ReactionSieve(model_container, desired_desc_dict)
    
  def recommend(self):
    rxns = self.generator.generate()
    desired_rxns = self.sieve.filter(rxns)
    return desired_rxns

In [19]:
from DRP.models import ModelContainer

In [20]:
from DRP.models import 

SyntaxError: invalid syntax (<ipython-input-20-ad5f595b6914>, line 1)

In [ ]:
a = ReactionSieve(ModelContainer.objects.get(pk=1), ReactionGenerator(), desired_desc_dict)

In [21]:
from DRP.models import NumRxnDescriptor

In [22]:
NumRxnDescriptor.objects.all()

[<NumRxnDescriptor: Reaction temperature in degrees C>, <NumRxnDescriptor: Reaction time in minutes>, <NumRxnDescriptor: Reaction pH>, <NumRxnDescriptor: Pre heat standing time>, <NumRxnDescriptor: Mols of K in a reaction.>, <NumRxnDescriptor: Mols of Yb in a reaction.>, <NumRxnDescriptor: Mols of As in a reaction.>, <NumRxnDescriptor: Mols of Xe in a reaction.>, <NumRxnDescriptor: Mols of Ar in a reaction.>, <NumRxnDescriptor: Mols of Hg in a reaction.>, <NumRxnDescriptor: Mols of Co in a reaction.>, <NumRxnDescriptor: Mols of Pb in a reaction.>, <NumRxnDescriptor: Mols of Si in a reaction.>, <NumRxnDescriptor: Mols of Rb in a reaction.>, <NumRxnDescriptor: Mols of Ba in a reaction.>, <NumRxnDescriptor: Mols of Na in a reaction.>, <NumRxnDescriptor: Mols of Po in a reaction.>, <NumRxnDescriptor: Mols of F in a reaction.>, <NumRxnDescriptor: Mols of Ir in a reaction.>, <NumRxnDescriptor: Mols of Eu in a reaction.>, '...(remaining elements truncated)...']